**Dependencies:**

- To direct c++ std stream to Jupyter notebook:
    `pip install wurlitzer` (python 2.7 only?)

- Deformed frame visualization
    Install `meshcat-python`: https://github.com/rdeits/meshcat-python

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import numpy as np
import pyconmech as cm
# from wurlitzer import sys_pipes

import meshcat

import meshcat.geometry as g
from deformed_frame_viz import meshcat_visualize_deformed

In [4]:
vis = meshcat.Visualizer()
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


# check deformation for a completed structure

In [7]:
import os
import time
from deformed_frame_viz import meshcat_visualize_deformed
cwd = os.getcwd()

# json_path = os.path.join(cwd, "assembly_instances/extrusion","fertility.json")
# json_path = os.path.join(cwd, "assembly_instances/extrusion","topopt-100_S1_03-14-2019_w_layer.json")
json_path = os.path.join(cwd, "assembly_instances/extrusion","rotated_dented_cube.json")

vis.delete()
disc = 10
exagg_ratio = 1.0
time_step = 1.0
scale = 1.0

# with sys_pipes():

sc = cm.stiffness_checker(json_file_path = json_path, verbose = False)
#     sc.set_output_json(True)
#     sc.set_output_json_path(file_path = cwd, file_name = "sf-test_result.json")

# manually assigned point loads
#     ext_load = np.zeros([1,7])
#     ext_load[0,0] = 3
#     ext_load[0,4] = -1
#     include_sw = True

# import load from a json file
#     ext_load, include_sw = cm.parse_load_case_from_json(load_case_path)
#     print('external load:')
#     print(ext_load)
#     print('include self weight : {0}'.format(include_sw))
#     sc.set_load(nodal_forces = ext_load)
#     sc.set_self_weight_load(include_sw) # now this is true by default

#     sc.set_nodal_displacement_tol(transl_tol=1e-3)

# existing_ids = [125, 126, 115, 122, 111, 108, 23, 22, 98, 75, 64, 34, 61, 65, 59, 60, 39, 36, 44, 67]
#     existing_ids = [64] # unsupported

# print("Pass criteria? {0}\n".format(sc.solve(list(existing_ids))))
print("Pass criteria? {0}\n\n".format(sc.solve()))

# Collecting results
success, nD, fR, eR = sc.get_solved_results()
print("pass criteria?\n {0}".format(success))

trans_tol, rot_tol = sc.get_nodal_deformation_tol()

max_trans, max_rot, max_trans_vid, max_rot_vid = sc.get_max_nodal_deformation()
compliance = sc.get_compliance()

print('max deformation: translation: {0} / tol {1}, at node #{2}'.format(max_trans, trans_tol, max_trans_vid))
print('max deformation: rotation: {0} / tol {1}, at node #{2}'.format(max_rot, rot_tol, max_rot_vid))

# e_stiffness_mats = sc.get_element_stiffness_matrices(exist_ids=[])
e_stiffness_mats = sc.get_element_stiffness_matrices()
# print(e_stiffness_mats[0].shape)

e_rot_mats = sc.get_element_local2global_rot_matrices()
# print(e_rot_mats[0].shape)

# nodal_load = sc.get_nodal_load(existing_ids)
nodal_load = sc.get_nodal_load([])
# print(nodal_load.size)

e2dof_map = sc.get_element2dof_id_map()
# print(e2dof_map.shape)

v2dof_map = sc.get_node2dof_id_map()
# print(v2dof_map.shape)
# print(v2dof_map)

num_node, num_e = sc.get_frame_stat()
print('# of nodes: {}. # of elements: {}'.format(num_node, num_e))

# visualize deformed structure
orig_beam_shape = sc.get_original_shape(disc=disc, draw_full_shape=False)

cp_orig_beam_shape = orig_beam_shape
# print(cp_orig_beam_shape)

beam_disp = sc.get_deformed_shape(exagg_ratio=exagg_ratio, disc=disc)
# print(beam_disp)

meshcat_visualize_deformed(vis, beam_disp, cp_orig_beam_shape, disc=disc, scale=scale)

Pass criteria? False


pass criteria?
 False
max deformation: translation: 0.0015263557142933034 / tol 0.001, at node #138
max deformation: rotation: 0.006388599974502525 / tol 0.05233333333333334, at node #2
# of nodes: 168. # of elements: 332


# animate deformation for a construction sequence

In [22]:
import os
import time
cwd = os.getcwd()

json_path = os.path.join(cwd,"tower_3D.json")
# json_path = os.path.join(cwd,"sf-test_3-frame.json")
load_case_path = os.path.join(cwd,"tower_3D_load_case.json")

vis.delete()
disc = 10
exagg_ratio=1.0
time_step = 1.0

sc = cm.stiffness_checker(json_file_path = json_path, verbose = False)

#     sc.set_output_json(True)
#     sc.set_output_json_path(file_path = cwd, file_name = "sf-test_result.json")

#     ext_load = np.zeros([1,7])
#     ext_load[0,0] = 3
#     include_sw = False

#     ext_load, include_sw = cm.parse_load_case_from_json(load_case_path)
#     print('external load:')
#     print(ext_load)
#     print('include self weight : {0}'.format(include_sw))
#     sc.set_load(nodal_forces = ext_load)
#     sc.set_self_weight_load(include_sw)

sc.set_self_weight_load(True)

for i in range(0,24):
    vis.delete()
    existing_ids = list(range(0,i+1))
    sc.solve(existing_ids)

    orig_beam_shape = sc.get_original_shape(disc=disc, draw_full_shape=False)
    beam_disp = sc.get_deformed_shape(exagg_ratio=exagg_ratio, disc=disc)
    meshcat_visualize_deformed(vis, beam_disp, orig_beam_shape, disc=disc, scale=0.5)
    time.sleep(time_step)

# repetitive run test

In [11]:
import os
import time
cwd = os.getcwd()

json_path = os.path.join(cwd,'assembly_instances','extrusion', 'topopt-100_S1_03-14-2019_w_layer.json')
N = 10000

solve_partial = True
existing_ids = [125, 126, 115, 122, 111, 108, 23, 22, 98, 75, 64, 34, 61, 65, 59, 60, 39, 36, 44, 67]

check_result = False

## without reinit on everal `solve` call

In [12]:
sc = cm.stiffness_checker(json_file_path = json_path, verbose = False)

st_time = time.time()
for i in range(0,N):
    if solve_partial:
        sc.solve(existing_ids)
    else:
        sc.solve()

    if check_result:
        trans_tol, rot_tol = sc.get_nodal_deformation_tol()
        max_trans, max_rot, max_trans_vid, max_rot_vid = sc.get_max_nodal_deformation()
        compliance = sc.get_compliance()
        print('iter - {0}'.format(i))
        print('max deformation: translation : {0} / tol {1}, at node #{2}'.format(max_trans, trans_tol, max_trans_vid))
        print('max deformation: rotation : {0} / tol {1}, at node #{2}'.format(max_rot, rot_tol, max_rot_vid))

print('avg time: {0} s'.format((time.time() - st_time) / N))

avg time: 0.0003613334655761719 s


## reinit on everal `solve` call

In [14]:
st_time = time.time()

for i in range(0,N):
    sc = cm.stiffness_checker(json_file_path = json_path, verbose = False)

    existing_ids = list(range(0,10))
    if solve_partial:
        sc.solve(existing_ids)
    else:
        sc.solve()

    if check_result:
        trans_tol, rot_tol = sc.get_nodal_deformation_tol()
        max_trans, max_rot, max_trans_vid, max_rot_vid = sc.get_max_nodal_deformation()
        compliance = sc.get_compliance()
        print('iter - {0}'.format(i))
        print('max deformation: translation : {0} / tol {1}, at node #{2}'.format(max_trans, trans_tol, max_trans_vid))
        print('max deformation: rotation : {0} / tol {1}, at node #{2}'.format(max_rot, rot_tol, max_rot_vid))

print('avg time: {0} s'.format((time.time() - st_time) / N))

avg time: 0.001588550329208374 s


## What exactly is the `stiffness_checker` trying to solve?

Elastic structure deforms under load, and by deforming themselves they develop resistance (or reaction) force to balance the external load.

So in a nutshell, the stiffness checker calculates elastic deformation and corresponding reaction force of a frame structure under given load. We mainly consider the gravity load induced by elements' self-weight in construction sequencing.

Conceptually, the solver tries to piece many elements' unit behavior together to reach equilibrium with the external force. Each element obeys both Hooke's law and the [Beam equations](https://en.wikipedia.org/wiki/Euler%E2%80%93Bernoulli_beam_theory), which tells us how does an element **develops internal force** to balance external load via **deformation**.

So locally in **each beam's own local coordinate system**, we have the local elastic equation:
$$\begin{pmatrix} F^{e-n1}_{L} \\ --- \\ F^{e-n2}_{L} \end{pmatrix} := \begin{pmatrix}F^{n1}_{Lx}\\ F^{n1}_{Ly} \\ F^{n1}_{Lz} \\ M^{n1}_{Lx} \\ M^{n1}_{Ly} \\ M^{n1}_{Lz} \\ --- \\ F^{n2}_{Lx}\\ F^{n2}_{Ly} \\ F^{n2}_{Lz} \\ M^{n2}_{Lx} \\ M^{n2}_{Ly} \\ M^{n2}_{Lz}\end{pmatrix} = \mathbf{K_e} \begin{pmatrix} d^{n1}_{Lx}\\ d^{n1}_{Ly} \\ d^{n1}_{Lz} \\ \theta^{n1}_{Lx} \\ \theta^{n1}_{Ly} \\ \theta^{n1}_{Lz} \\ --- \\ d^{n2}_{Lx}\\ d^{n2}_{Ly} \\ d^{n2}_{Lz} \\ \theta^{n2}_{Lx} \\ \theta^{n2}_{Ly} \\ \theta^{n2}_{Lz} \end{pmatrix} = \mathbf{K_e} \begin{pmatrix} u^{e-n1}_{L} \\ --- \\ u^{e-n2}_{L} \end{pmatrix}$$

Here you can conceptually think about this $12 \times 12$ element stiffness matrix $\mathbf{K_e}$ as the stiffness factor $k$ in Hooke's law $\Delta{F} = k \Delta{x}$ in a string system. The only difference is that it's capturing the shear, bending, and torsion effect as well, not only the axial elongation (see picture below).

![frame_equation](../../../../docs/images/frame_equation.png)
image source: [MIT 1.571 lecture note 11](../../../../docs/references/MIT_1.571_L11_Displacement_Method.pdf), page 11 (Pierre Ghisbain)

But since some elements are sharing a node, these elements' reaction must relate to each other so that 
1. the deformation at the shared node ($u^{e-v}_{L}$) is the same
2. the reaction forces of these elements ($F^{e-v}_{L}$) reach equilibrium at the shared node.

Thus, at each node $v$, we have first the equilibrium equation:

$$\sum_{e \in \{e | e \sim v\}} \begin{pmatrix}R_{e, GL} & 0 \\ 0 & R_{e, GL}\\\end{pmatrix}\begin{pmatrix}F^{v}_{e, Lx} \\ F^{v}_{e, Ly} \\ F^{v}_{e, Lz} \\ M^{v}_{e, Lx} \\ M^{v}_{e, Lx} \\ M^{v}_{e, Lz} \end{pmatrix} = \sum_{e \in \{e | e \sim v\}} (F_{v, \textrm{e self-w load}}) + F_{v, \textrm{pt load}} $$

The RHS of the equation above represents all the loads (gravity, external point load) at node $v$. I will come back to loads in the section "The relationship between `fixities_reaction` and `element_reaction`" below. Notice that we have to apply this local to global rotation matrix to transform all the element internal force to the global coordinate system.

Then, we also have to make sure the shared nodal deformation is the same for all the connected elements, so by plugging into the local elastic equation above, the equilibrium equation becomes:

$$(\sum_{e \in \{e | e \sim v\}} \begin{pmatrix}R_{e, GL} & 0 \\ 0 & R_{e, GL}\\\end{pmatrix} \mathbf{K_e} \begin{pmatrix}R_{e, GL} & 0 \\ 0 & R_{e, GL}\\\end{pmatrix}^T) \begin{pmatrix} u^{e-n1}_{G} \\ u^{e-n2}_{G} \end{pmatrix} = \sum_{e \in \{e | e \sim v\}} (F_{v, \textrm{e self-w load}}) + F_{v, \textrm{pt load}} $$

Notice that we are enforcing the nodal deformation compatibility by having all the connected elements share the same nodal deformation $\begin{pmatrix} u^{e-n1}_{G}, u^{e-n2}_{G} \end{pmatrix}$ **in global coordinate**.

The equation above must be satisfied by all the nodes in the structure, and we have to solve all of them together by *assembling* the global stiffness matrix.

PS: we can really see the essence of FEM here: first we have the physics model for one single element, then we try to enforce (1) internal reaction equibilirium at nodes (shared element boundary) (2) compability on the deformation at the shared element boundary. Finally, we assembly these nodal equations together into a giant linear system and we solve.

# `stiffness_checker` outputs explained

We have three outputs from the `stiffness_checker`:

## 1.`nodal_deformation`

- (num_of_nodes x 7) matrix

Each row is $(\textrm{node id}, d_{Gx}, d_{Gy}, d_{Gz}, \theta_{Gx}, \theta_{Gy}, \theta_{Gz})$. Here the subscript $G\,\cdot$ means the displacements are described in the **global coordinate system**.

## 2. `element_reaction`

- (num_of_elements x 13) matrix

Each row is 
$(\textrm{element_id}, F^{n1}_{Lx}, F^{n1}_{Ly}, F^{n1}_{Lz}, M^{n1}_{Lx}, M^{n1}_{Ly}, M^{n1}_{Lz}, F^{n2}_{Lx}, F^{n2}_{Ly}, F^{n2}_{Lz}, M^{n2}_{Lx}, M^{n2}_{Ly}, M^{n2}_{Lz})$. Each row describes the element's internal reaction force and moment at the two end points.

Here $n1, n2$ refer to the end nodes of this element. $F_{L\,\cdot}$ and $M_{L\,\cdot}$ refer to the element's internal force and moment **under the element's local coordinate system**. 

By convention, the local cooridinate system is constructed by setting the origin to node $n1$, and the local x axis as the direction from $n1$ to $n2$ (see picture below). The local y axis is constructed by simply checking if the element's local x axis is aligned with the global z axis, and construct the local y axis by taking cross product between the local x axis and the global z. See the [src code](https://github.com/yijiangh/conmech/blob/224b24b07688af61033994a23f24fb9bb0e7c2d0/src/stiffness_checker/Util.cpp#L72-L142) for more details. 

PS: This extra degree of freedom in choosing local y axis makes it very hard to compare our internal element reaction results with an existing FEM solver...

![frame_local_coordinate](../../../../docs/images/frame_element_local_coordiante.jpg)
image source: https://www.sciencedirect.com/topics/engineering/moment-distribution

## 3.`fixities_reaction`
- (num_of_fixities x 7) matrix

Each row is $(\textrm{fix_node_id}, F_{Gx}, F_{Gy}, F_{Gz}, M_{Gx}, M_{Gy}, M_{Gz})$. Here the forces and moments are described in **global coordinate system**.

## The relationship between `fixities_reaction` and `element_reaction`

Since we are dealing with fully constrained elastic frame structures (all the frame nodes and fixities are 6-dof fixed), a partially assembled structure will never collapse because of mechanism, as long as it does not have unconnected, floating elements. Or in other words, reaction force will be in equilibrium at each node (no matter they are grounded or not).

### For nodes that are not fixities

For nodes that are not fixities, the forces are from connected elements' internal reaction force and loads. So the force equilibrium at this node $v$ is:
$$\sum_{e \in \{e | e \sim v\}} \begin{pmatrix}R_{e, GL} & 0 \\ 0 & R_{e, GL}\\\end{pmatrix}\begin{pmatrix}F^{v}_{e, Lx} \\ F^{v}_{e, Ly} \\ F^{v}_{e, Lz} \\ M^{v}_{e, Lx} \\ M^{v}_{e, Lx} \\ M^{v}_{e, Lz} \end{pmatrix} = \sum_{e \in \{e | e \sim v\}} (F_{v, \textrm{e self-w load}}) + F_{v, \textrm{pt load}} $$

Where $R_{e, GL}$ is the $3 \times 3$ global to local rotation matrix of element $e$. The element internal reaction $\begin{pmatrix}F^{v}_{e, Lx}, F^{v}_{e, Ly}, F^{v}_{e, Lz}, M^{v}_{e, Lx}, M^{v}_{e, Lx}, M^{v}_{e, Lz} \end{pmatrix}$ is the entries that we get from element $e$'s corresponding row in `element_reaction`. This vector is descirbed in local coordinate of element $e$, and that's why we need to use the rotation matrix $R_{e, GL}$ to transform it back to global coordinate, so that we can have all the element's reaction force described in the same system.

Notice that each row of `element_reaction` contains two such 6 dimensional vectors, corresponding to two end point of element $e$. 
(We do not have a convienient API to help us know which node index these two 6-dim vectors are referring to now. But I can add one if we need it.)


The force $F_{v, \textrm{e self-w load}}$ is the lumped self-weight of the element $e$ described in the global coordinate. We will talk about this formation of this self-weight load in more detail in the next section. 

The $F_{v, \textrm{pt load}}$ refers to extra loads that are specified on the node. We don't have this in our `pb-construction` sequencing context (but "classic structural analysis" usually works with these point load scenarios!)

The summation over all the connected element $e \in \{e | e \sim v\}$ means that both the number of internal force and the self-weight load will be changing when we are considering different partially assembled structures. Indeed, within the implementation of `stiffness_checker`, we pre-calculate all the [element-wise stiffness matrix](https://github.com/yijiangh/conmech/blob/224b24b07688af61033994a23f24fb9bb0e7c2d0/src/stiffness_checker/Stiffness.cpp#L246) and [self-weight load](https://github.com/yijiangh/conmech/blob/224b24b07688af61033994a23f24fb9bb0e7c2d0/src/stiffness_checker/Stiffness.cpp#L363), both of which are in global coordinate. Whenever we feed the checker with a set of exisiting element ids, we simply assembly corresponding element's stiffness matrix and load vector together to form a structure-wise linear system and solve it. By doing this, we are not doing repetitive calculation.

### For nodes that are fixities

For nodes that are fixities, the forces are from connected elements' internal reaction force and the reaction force from the ground (fixities). So the force equilibrium at this node $v$ is:

$$\sum_{e \in \{e | e \sim v\}} \begin{pmatrix}R_{e, GL} & 0 \\ 0 & R_{e, GL}\\\end{pmatrix}\begin{pmatrix}F^{v}_{e, Lx} \\ F^{v}_{e, Ly} \\ F^{v}_{e, Lz} \\ M^{v}_{e, Lx} \\ M^{v}_{e, Lx} \\ M^{v}_{e, Lz} \end{pmatrix} = \begin{pmatrix}RF^{v}_{Gx} \\ RF^{v}_{Gy} \\ RF^{v}_{Gz} \\ RM^{v}_{Gx} \\ RM^{v}_{Gx} \\ RM^{v}_{Gz} \end{pmatrix}$$

Where $R_{e, GL}$ is the $3 \times 3$ global to local rotation matrix of element $e$. Here the $RF^{v}_{G\cdot}$ and $RM^{v}_{G\cdot}$ refer to the fixity reaction force and moment in the global coordinate, **which is specified by the corresponding row that we get from the output matrix `fixities_reaction`**.

## Calculation of self-weight load at a node

The main load case of construction sequencing is self-weight, i.e. the gravity force that acts *along the span of the frame element*, which is a distributed load. One common idea in Finite Element Analysis (FEM) is to "lump" these distributed load to its boundary. In our context, the boundary of our elements is two end points, so we calculate the equivalent point load acting at the element end points.

From the beam equation, we can calculate the equivalent point loads (force, moment) for a uniform load of density $w$ using the formula outlined in the picture below:

![fixed_end_unif_loaded_beam](../../../../docs/images/fixed_ends_uniform_loaded_beam.png)
image source: [MIT 1.571 lecture note 11](../../../../docs/references/MIT_1.571_L11_Displacement_Method.pdf), page 9 (Pierre Ghisbain)

And for skewed elements, where the local x axis is not aligned with the global z axis (- gravity direction), we can decompose the gravity force to components that are perpendicular to the local x axis $P_{\perp x}$ amd $P_{// x}$. The $P_{// x}$ can be simply treated as load along the local x axis, and $P_{\perp x}$ can be lumped into end point loads using the fixed-end uniformly loaded beam formula above.

This is a still a bit handwavy for now, but I will find time to fill in more details on this later... See the [source code](https://github.com/yijiangh/conmech/blob/224b24b07688af61033994a23f24fb9bb0e7c2d0/src/stiffness_checker/Stiffness.cpp#L363) for more details.